In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

#Load the dataset from the file
train = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")

#Print the shape of the data
train.shape, test.shape

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/equity-post-HCT-survival-predictions/train.csv'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

#Load the dataset from the file
train = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")

#Print the shape of the data
train.shape, test.shape

In [3]:
#Create two new columns for XGBoost regressor
train["y"] = train["efs"]
# Separate numerical and categorical columns (XGBoost can handle NAN values)
numerical_cols = train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns
# Fill Nan of the train and test data
train[categorical_cols] = train[categorical_cols].fillna("NAN")
test[categorical_cols] = test[categorical_cols].fillna("NAN")

In [4]:
labelencoder = LabelEncoder()
#Label encoding both train and test set
for cols in categorical_cols:
    train[cols] = labelencoder.fit_transform(train[cols])
    test[cols] = labelencoder.fit_transform(test[cols])
# Self-made standardized dataset
def standardize_with_nan(df):
    return (df - df.mean()) / df.std()
train_standardized = train.apply(standardize_with_nan, axis=0)
test_standardized = test.apply(standardize_with_nan, axis=0)

In [5]:
all_cols = list(train.columns)
all_cols.remove("efs")
all_cols.remove("efs_time")
all_cols.remove("ID")
all_cols.remove("y")

In [6]:
# Cross-K validation test
from sklearn.model_selection import KFold
folds = 5
kf = KFold(n_splits=folds, shuffle=True, random_state=42)

OOF = np.zeros(len(train))
pred = np.zeros(len(test))

for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train["y"])):

    print(f"Training fold {fold + 1}")
    X_train = train.loc[train_idx, all_cols].copy()
    y_train = train.loc[train_idx, "y"]
    X_valid = train.loc[valid_idx, all_cols].copy()
    y_valid = train.loc[valid_idx,"y"]
    X_test = test[all_cols].copy()

    # Train model (example: XGBRegressor for simplicity)
    model = XGBRegressor(
        max_depth=5,
        learning_rate=0.08,
        n_estimators=1000,
        eval_metric="mae",
        early_stopping_rounds=40,
        objective="reg:squarederror"
    )
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    OOF[valid_idx] = model.predict(X_valid)
    pred += model.predict(test[all_cols])

pred /= folds

Training fold 1
[0]	validation_0-mae:0.49230
[100]	validation_0-mae:0.41025
[200]	validation_0-mae:0.40321
[300]	validation_0-mae:0.40067
[400]	validation_0-mae:0.39931
[462]	validation_0-mae:0.39949
Training fold 2
[0]	validation_0-mae:0.49129
[100]	validation_0-mae:0.40681
[200]	validation_0-mae:0.39911
[300]	validation_0-mae:0.39725
[400]	validation_0-mae:0.39630
[441]	validation_0-mae:0.39621
Training fold 3
[0]	validation_0-mae:0.49204
[100]	validation_0-mae:0.40892
[200]	validation_0-mae:0.40207
[300]	validation_0-mae:0.39981
[400]	validation_0-mae:0.39875
[500]	validation_0-mae:0.39811
[600]	validation_0-mae:0.39781
[603]	validation_0-mae:0.39784
Training fold 4
[0]	validation_0-mae:0.49257
[100]	validation_0-mae:0.41228
[200]	validation_0-mae:0.40610
[300]	validation_0-mae:0.40360
[400]	validation_0-mae:0.40241
[500]	validation_0-mae:0.40123
[600]	validation_0-mae:0.40098
[628]	validation_0-mae:0.40109
Training fold 5
[0]	validation_0-mae:0.49235
[100]	validation_0-mae:0.41380


In [7]:
sub = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv")
sub["prediction"] = -pred
print("Submission shape:",sub.shape)
sub.to_csv("submission.csv", index=False)

Submission shape: (3, 2)
